In [1]:
import numpy as np 
import random 
import re 
import tensorflow as tf 
from sklearn.model_selection import train_test_split 
from sklearn import preprocessing 


In [2]:
positive_training_data = "positive_training_dataset.txt"
negative_training_data = "negative_training_dataset.txt" 
positive_testing_data = "positive_testing_dataset.txt" 
negative_testing_data = "negative_testing_dataset.txt"#数据集的路径

In [3]:
def RNA_to_feature(str_rna, str_structure): 
#此函数将输入的RNA及其二级结构转化为输入的特征值 
    # 接受两个字符串,第一个字符串是基因序列,第二个字符串是对应的二级结构 
#     print(str_rna,str_structure)
#     energy = re.findall(r'-{0,1}\d{1,2}.\d{1,2}', str_structure)[0] 
    
    #利用正则表达式提取自由能 
    rna, structure = '','' 
    
    #初始化RNA和二级结构的数组 
    for i in range(len(str_rna)): 
        #这个循环用来保证二级结构和编码同样长度，同时避免混入非RNA编码 
        if str_rna[i] is not 'A' and str_rna[i] is not 'C' and str_rna[i] is not 'G' and str_rna[i] is not 'U': 
            continue 
        rna += str_rna[i] 
        structure += str_structure[i] 
        structure2int = [0 for i in range(len(structure))] 
        for i in range(len(structure)): 
        #这个循环将二级结构的编码转换为0，1编码，方便三元组编码的运用 
            if structure[i] is '(' or ')': 
                structure2int[i]= 1 
            if structure[i] is '.': 
                structure2int[i]= 0 
    #接下来构造一个32维特征，用于储存三元组编码方式的特征值，以字典形式给出。并初始化为0。
    rna_dic = {'A000': 0, 'A001': 0, 'A010': 0, 'A011': 0, 'A100': 0,'A101': 0, 'A110': 0, 'A111': 0, 'C000': 0, 'C001': 0, 'C010': 0, 'C011': 0, 'C100': 0,'C101': 0, 'C110': 0, 'C111': 0, 'G000': 0, 'G001': 0, 'G010': 0, 'G011': 0, 'G100': 0,'G101': 0, 'G110': 0, 'G111': 0, 'U000': 0, 'U001': 0, 'U010': 0, 'U011': 0, 'U100': 0,'U101': 0, 'U110': 0, 'U111': 0} 
    #然后搭建一个特征数组，方便判断使用。 
    rna_index = ['A000', 'A001', 'A010', 'A011', 'A100','A101', 'A110', 'A111', 'C000', 'C001', 'C010', 'C011', 'C100','C101', 'C110', 'C111', 'G000', 'G001', 'G010', 'G011', 'G100','G101', 'G110', 'G111', 'U000', 'U001', 'U010', 'U011', 'U100','U101', 'U110', 'U111'] 
    for i in range(len(rna)-2): 
        str2all = rna[i]+ __builtins__.str(structure2int[i])+ __builtins__.str(structure2int[i+1])+ __builtins__.str(structure2int[i+2]) 
        rna_dic[str2all] += 1 
    formalInput = [0 for i in range(32)] 
    #初始化最终特征向量formalInput，作为最后返回的特征向量。
    count = 0; 
    for i in rna_index: 
        formalInput[count] = rna_dic[i] 
        count += 1
#     print(energy)
#     formalInput[32] = float(energy) 
    #print(formalInput)
    #最后，将formalInput作为最终特征向量返回。 
    return formalInput
    
    
    
    

In [4]:
def input_form_x(data_path): 
#数据读取函数 
    with open(data_path) as data: 
        #打开文件 
        
        form_x = [] 
        fp = data.readlines() 
        print(len(fp))
        for i in range(0, len(fp), 3): 
            #每次跳两个读取 
            
            form_x.append(RNA_to_feature(fp[i + 1], fp[i + 2])) 
            #将奇数行（RNA编码）和偶数行（二级结构）一同输入，一起进行转换
            
    return form_x



In [5]:



def data_process(): 
    X_train_correct = input_form_x(positive_training_data) 
    # 获取数据 
    Y_train_correct = [] 
    for i in range(len(X_train_correct)): 
        Y_train_correct.append([1,0]) 
        
    X_train_wrong = input_form_x(negative_training_data) 
    Y_train_wrong = [] 
    for i in range(len(X_train_wrong)):
        Y_train_wrong.append([0,1])
    X_test_correct = input_form_x(positive_testing_data) # 获取数据 
    Y_test_correct = [] 
    for i in range(len(X_test_correct)): 
        Y_test_correct.append([1,0]) 
    X_test_wrong = input_form_x(negative_testing_data) 
    Y_test_wrong = [] 
    for i in range(len(X_test_wrong)): 
        Y_test_wrong.append([0,1])#采取独热编码进行二分类，是 与 不是。 
    return X_train_correct,Y_train_correct,X_train_wrong ,Y_train_wrong ,X_test_correct ,Y_test_correct ,X_test_wrong ,Y_test_wrong
    #然后将所有数据返回。
     

NameError: name 'plt' is not defined

In [10]:
#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=1,seed=1) #生成一个截断的正态分布
    return tf.Variable(initial)
    #return tf.Variable(tf.zeros(shape))

#初始化偏置
def bias_variable(shape):
    initial = tf.truncated_normal(shape,stddev=1,seed=1)
    return tf.Variable(initial)
    #return tf.Variable(tf.zeros(shape))

#卷积层
def conv2d(x,W):
    #input tensor of shape [batch, in_height, in_width, in_channels]
    #filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #strides[0]=strides[3]=1. strides[1]代表ｘ方向的步长，strids[2]代表ｙ方向的步长
    #padding: A string from "SAME", "VALID"
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

#池化层
def max_pool(x):
    #ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')



def multilayer_perceptron(x_train, x_test, y_train, y_test): 


    x_train_size = np.size(x_train, 0)
    #定义两个placeholder
    x = tf.placeholder(tf.float32, [None, 32]) 
    y = tf.placeholder(tf.float32, [None, 2])
    
    #改变x的格式转为４Ｄ的向量【batch, in_height, in_width, in_channels]
    x_image = tf.reshape(x,[-1, 1, 32, 1])

    #初始化第一个卷积层的权值和偏量
    W_conv1 = weight_variable([1,3,1,64])#5*5的采样窗口，３２个卷积核从4个平面抽取特征
    b_conv1 = bias_variable([64])#每一个卷积核一个偏置值

    #把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool(h_conv1)#进行max-pooling,12-by-40
    
#     #初始化第二个卷积层的权值和偏置
#     W_conv2 = weight_variable([2,2,32,64]) #5*5的采样窗口，64个卷积核从32个平面抽取特征
#     b_conv2 = bias_variable([64]) #每一个卷积核一个偏置值

#     #把H_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
#     h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
#     h_pool2 = max_pool(h_conv2)#6-by-5
    

    #23*20的图片第一次卷积后还是23*20,第一次池化后变为12*10
    #第二次卷积后为12*10,第二次池化后变为6*5
    #进过上面操作后得到64张6*5的平面

    #初始化第一全链接层的权值
    W_fc1 = weight_variable([1*16*64,512]) #上一层有6*10*64个神经元,全连接层有1024个神经元
    b_fc1 = bias_variable([512])

    #把池化层2的输出扁平化为1维
    h_pool2_flat = tf.reshape(h_pool1,[-1,1*16*64])
    #求第一个全连接层的输出
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    #keep_prob用了表示神经元的输出概率
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    #初始化第二个全连接层
    W_fc2 = weight_variable([512,2])
    b_fc2 = bias_variable([2])

    
    #计算输出
    prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)
    # 结果存放在一个布尔列表中
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    
    #交叉熵代价函数
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
    
    #使用AdamOptimizer进行优化
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    #求准确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))




    with tf.Session() as sess: 
        sess.run(tf.global_variables_initializer())
        #初始化参数 
        # Training cycle 
        for epoch in range(10000): 
            batch_index = random.randint(0, 1000) 
            #每一次从0-1000随机选出一个数，作为起始点 
            batch_data = x_train[batch_index:batch_index + 32] 
            batch_labels = y_train[batch_index:batch_index + 32] 
            #在上面所选的起始点开始，拿出32组数据进行一次训练 
            
            sess.run(optimizer,feed_dict={x: batch_data, y: batch_labels, keep_prob: 0.9})
            
            
            #进行训练 
            if epoch%300 == 0: 
                print(epoch)
                #accuracy = sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0})
                print("Accuracy:", sess.run(accuracy, {x:x_test,y: y_test, keep_prob: 1.0}))
                print("Loss:", sess.run(cost, {x:x_test,y: y_test, keep_prob: 1.0}))

                #每隔300次 输出对应的次数以及测试集所测试的准确度 
        # Test model 
     
        print("Total Accuracy:", accuracy.eval({x:x_test, y:y_test, keep_prob: 1.0})) 
        #最后，当训练结束时，输出最终的准确率。
        

In [7]:

# 训练集和测试集的随机分割
X_train_correct, Y_train_correct, X_train_wrong, Y_train_wrong, X_test_correct, Y_test_correct ,X_test_wrong, Y_test_wrong = data_process() 
#x_train, x_test, y_train, y_test = train_test_split(x_train + x_test, y_train + y_test, test_size=0.15,random_state=42) 
x_train, x_test, y_train, y_test = train_test_split(X_train_correct + X_train_wrong + X_test_correct + X_test_wrong,
                                                    Y_train_correct + Y_train_wrong + Y_test_correct + Y_test_wrong,
                                                    test_size=0.15,random_state=42) 



1452
1452
372
372


In [11]:
#学习率设置
learning_rate = 1e-4

multilayer_perceptron(x_train, x_test, y_train, y_test)

0
Accuracy: 0.4863388
Loss: 0.82692283
300
Accuracy: 0.58469945
Loss: 0.7285625
600
Accuracy: 0.61202186
Loss: 0.70123976
900
Accuracy: 0.568306
Loss: 0.7449493
1200
Accuracy: 0.73224044
Loss: 0.5810233
1500
Accuracy: 0.75956285
Loss: 0.5542135
1800
Accuracy: 0.72677594
Loss: 0.5864856
2100
Accuracy: 0.8251366
Loss: 0.4881251
2400
Accuracy: 0.78688526
Loss: 0.5249299
2700
Accuracy: 0.7978142
Loss: 0.5154474
3000
Accuracy: 0.8306011
Loss: 0.48265925
3300
Accuracy: 0.8196721
Loss: 0.49267724
3600
Accuracy: 0.8469945
Loss: 0.4631627
3900
Accuracy: 0.84153
Loss: 0.47173154
4200
Accuracy: 0.863388
Loss: 0.44987372
4500
Accuracy: 0.87431693
Loss: 0.43894467
4800
Accuracy: 0.8579235
Loss: 0.45533812
5100
Accuracy: 0.863388
Loss: 0.44980568
5400
Accuracy: 0.863388
Loss: 0.44987354
5700
Accuracy: 0.87431693
Loss: 0.43894467
6000
Accuracy: 0.87431693
Loss: 0.4389428
6300
Accuracy: 0.87431693
Loss: 0.43894467
6600
Accuracy: 0.8469945
Loss: 0.4662672
6900
Accuracy: 0.87431693
Loss: 0.43869698
7200

In [9]:
# print(len(x_train),len(y_train))
# print(len(x_test),len(y_test))
# print(y_test)